In [39]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [40]:
def read_dataset():
    df = pd.read_csv('sonar.all-data.csv')
    
    X = df[df.columns[:60]].values
    y = df[df.columns[60]]
    
    #Encode the dependent variable
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    Y = one_hot_encode(y)
    print(X.shape)
    return (X,Y)

In [41]:
def one_hot_encode(labels):
    n_lables = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_lables,n_unique_labels))
    one_hot_encode[np.arange(n_lables),labels] = 1
    return one_hot_encode


In [42]:
#Read Dataset
X,Y = read_dataset()

(207, 60)


In [43]:
#Shuffle the dataset to mixup the rows
X,Y = shuffle(X,Y,random_state=1)

In [44]:
#Convert dataset into train and test
train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size=0.2,train_size=0.8,random_state=415)

In [45]:
#Inspect the shape of train and test dataset
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)

(165, 60) (42, 60) (165, 2) (42, 2)


In [46]:
#Defining important parameter to work with tensor
learning_rate = 0.3
training_epoch = 1000

cost_history = np.empty(shape=[1],dtype=float)
n_dim = X.shape[1] #Number of columns
print("n_dim ",n_dim)

n_class = 2 #as we have mine and rock

model_path = r"/Users/nikhilyadav/Learning/Learning/Edureka_tf_Tuotorial/graph2_variable/NMI"


n_dim  60


In [47]:
#Define number of hidden layer and number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

x = tf.placeholder(tf.float32,[None,n_dim])
W = tf.Variable(tf.zeros([n_dim,n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32,[None,n_class])

print('x: ',x,'\nW:',W,'\nb: ',b,'\ny_: ',y_)

x:  Tensor("Placeholder_6:0", shape=(?, 60), dtype=float32) 
W: <tf.Variable 'Variable_16:0' shape=(60, 2) dtype=float32_ref> 
b:  <tf.Variable 'Variable_17:0' shape=(2,) dtype=float32_ref> 
y_:  Tensor("Placeholder_7:0", shape=(?, 2), dtype=float32)


In [48]:
#Define Model
def multilayer_perceptron(x,weights,bias):
    #Hidden layer 1 with sgimoid activation
    layer_1 = tf.add(tf.matmul(x,weights['h1']),bias['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    #hidden layer 2 with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),bias['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    
    #Hidden layer 3 with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']),bias['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    
    #Hidden layer 4 with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3,weights['h4']),bias['b4'])
    layer_4 = tf.nn.relu(layer_4)
    
    #Output layer with linear activation
    out_layer = tf.matmul(layer_4,weights['out']) + bias['out']
    return out_layer
    
    

    

In [49]:
#Define the weights and bias for each layer
weights = {
    'h1' : tf.Variable(tf.truncated_normal([n_dim,n_hidden_1])),
    'h2' : tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3' : tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4' : tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4,n_class]))
}

biases = {
    
    'b1' : tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2' : tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3' : tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4' : tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out' : tf.Variable(tf.truncated_normal([n_class])),

    
}

In [50]:
# Initialize all the variables

init = tf.global_variables_initializer()
saver = tf.train.Saver()


In [51]:
#Call your model defined
y = multilayer_perceptron(x,weights,biases)

In [52]:
#Define cost function and optimizers
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y,labels=y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [53]:
sess = tf.Session()
sess.run(init)

In [ ]:
#Calculate the cost and the accuracy for each epoch

mse_history = []
accuracy_history = []

for epoch in range(training_epoch):
    sess.run(training_step,feed_dict={x:train_x,y_:train_y})
    cost = sess.run(cost_function,feed_dict={x:train_x,y_:train_y})
    cost_history = np.append(cost_history,cost)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.arg_max(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    pred_y = sess.run(y,feed_dict={x:test_x})
    mse = tf.reduce_mean(tf.square(pred_y-test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy,feed_dict={x:train_x,y_:train_y}))
    accuracy_history.append(accuracy)
    
    print('Epoch: ',epoch,"- cost: ",cost,"- MSE: ",mse_,"- Train Accuracy: ",accuracy)
    

Epoch:  0 - cost:  121.52706 - MSE:  17840.157663495953 - Train Accuracy:  0.54545456
Epoch:  1 - cost:  5.455145 - MSE:  25.188088759615052 - Train Accuracy:  0.45454547
Epoch:  2 - cost:  13.0445595 - MSE:  207.2814678946594 - Train Accuracy:  0.54545456
Epoch:  3 - cost:  0.73004514 - MSE:  2.628226225470473 - Train Accuracy:  0.54545456
Epoch:  4 - cost:  0.68866515 - MSE:  2.68220791853946 - Train Accuracy:  0.53333336
Epoch:  5 - cost:  0.6879754 - MSE:  2.659597824830092 - Train Accuracy:  0.54545456
Epoch:  6 - cost:  0.6876561 - MSE:  2.649860126734384 - Train Accuracy:  0.53939396
Epoch:  7 - cost:  0.68740386 - MSE:  2.6451943369537116 - Train Accuracy:  0.53939396
Epoch:  8 - cost:  0.68717664 - MSE:  2.636862165479676 - Train Accuracy:  0.54545456
Epoch:  9 - cost:  0.6870504 - MSE:  2.638876392960529 - Train Accuracy:  0.54545456
Epoch:  10 - cost:  0.6869189 - MSE:  2.639157056343912 - Train Accuracy:  0.54545456
Epoch:  11 - cost:  0.68678087 - MSE:  2.6374855305954545 

Epoch:  95 - cost:  0.682634 - MSE:  2.7102608127092678 - Train Accuracy:  0.54545456
Epoch:  96 - cost:  0.6726245 - MSE:  2.6121394173675387 - Train Accuracy:  0.5939394
Epoch:  97 - cost:  0.66743594 - MSE:  2.6543205298441594 - Train Accuracy:  0.56363636
Epoch:  98 - cost:  0.66500884 - MSE:  2.6219900698940006 - Train Accuracy:  0.5939394
Epoch:  99 - cost:  0.66268545 - MSE:  2.627137715052806 - Train Accuracy:  0.55151516
Epoch:  100 - cost:  0.66208977 - MSE:  2.628867812742846 - Train Accuracy:  0.58181816
Epoch:  101 - cost:  0.66429734 - MSE:  2.6181825724044447 - Train Accuracy:  0.56363636
Epoch:  102 - cost:  0.6692618 - MSE:  2.638681541637213 - Train Accuracy:  0.56969696
Epoch:  103 - cost:  0.6828251 - MSE:  2.6531013488639794 - Train Accuracy:  0.55757576
Epoch:  104 - cost:  0.70049995 - MSE:  2.6153340676119714 - Train Accuracy:  0.47272727
Epoch:  105 - cost:  0.70489323 - MSE:  2.7361089640812857 - Train Accuracy:  0.54545456
Epoch:  106 - cost:  0.687154 - MSE:

Epoch:  190 - cost:  0.65646803 - MSE:  3.072216175096907 - Train Accuracy:  0.58181816
Epoch:  191 - cost:  0.6467362 - MSE:  3.1809083282655632 - Train Accuracy:  0.6484848
Epoch:  192 - cost:  0.64209706 - MSE:  3.043046198805452 - Train Accuracy:  0.6727273
Epoch:  193 - cost:  0.6370445 - MSE:  2.8911031036465666 - Train Accuracy:  0.6727273
Epoch:  194 - cost:  0.63238907 - MSE:  2.7244196464828607 - Train Accuracy:  0.6848485
Epoch:  195 - cost:  0.6266315 - MSE:  2.5468402536215082 - Train Accuracy:  0.6909091
Epoch:  196 - cost:  0.61987793 - MSE:  2.2636714405587703 - Train Accuracy:  0.7090909
Epoch:  197 - cost:  0.61518025 - MSE:  1.9512920342776643 - Train Accuracy:  0.6969697
Epoch:  198 - cost:  0.6126405 - MSE:  1.9151111805330858 - Train Accuracy:  0.7151515
Epoch:  199 - cost:  0.6115415 - MSE:  1.7637147084909874 - Train Accuracy:  0.6424242
Epoch:  200 - cost:  0.6161389 - MSE:  1.8672681176094177 - Train Accuracy:  0.6848485
Epoch:  201 - cost:  0.6444827 - MSE:  

Epoch:  284 - cost:  0.6172674 - MSE:  1.9015984567927764 - Train Accuracy:  0.6181818
Epoch:  285 - cost:  0.56545377 - MSE:  1.9690885059832388 - Train Accuracy:  0.73333335
Epoch:  286 - cost:  0.54546803 - MSE:  1.9077196514589811 - Train Accuracy:  0.6969697
Epoch:  287 - cost:  0.5346264 - MSE:  1.7454400546902842 - Train Accuracy:  0.72727275
Epoch:  288 - cost:  0.5273416 - MSE:  1.7148294675601796 - Train Accuracy:  0.72121215
Epoch:  289 - cost:  0.531985 - MSE:  1.5492248588743915 - Train Accuracy:  0.73333335
Epoch:  290 - cost:  0.6131065 - MSE:  2.0231808142240633 - Train Accuracy:  0.6606061
Epoch:  291 - cost:  1.0446241 - MSE:  1.9640414722450423 - Train Accuracy:  0.55151516
Epoch:  292 - cost:  0.6620695 - MSE:  1.8673495341422948 - Train Accuracy:  0.58787876
Epoch:  293 - cost:  0.62729543 - MSE:  1.6988586174757199 - Train Accuracy:  0.6484848
Epoch:  294 - cost:  0.5842027 - MSE:  1.4924934660563987 - Train Accuracy:  0.73939395
Epoch:  295 - cost:  0.5668755 - M

Epoch:  378 - cost:  0.57457596 - MSE:  1.2184954380289674 - Train Accuracy:  0.74545455
Epoch:  379 - cost:  0.5567614 - MSE:  1.1613017228687248 - Train Accuracy:  0.76363635
Epoch:  380 - cost:  0.53681904 - MSE:  1.0532448959710186 - Train Accuracy:  0.76969695
Epoch:  381 - cost:  0.5167294 - MSE:  1.0322394286382144 - Train Accuracy:  0.7818182
Epoch:  382 - cost:  0.49818513 - MSE:  0.9370004124412087 - Train Accuracy:  0.7818182
Epoch:  383 - cost:  0.48199186 - MSE:  0.9010536637030203 - Train Accuracy:  0.7878788
Epoch:  384 - cost:  0.4787993 - MSE:  0.8857631218639662 - Train Accuracy:  0.75757575
Epoch:  385 - cost:  0.55230445 - MSE:  1.0625785317111196 - Train Accuracy:  0.72121215
Epoch:  386 - cost:  1.0778526 - MSE:  2.0984990082504233 - Train Accuracy:  0.55757576
Epoch:  387 - cost:  0.5914968 - MSE:  1.5914971266939968 - Train Accuracy:  0.6787879
Epoch:  388 - cost:  0.6063661 - MSE:  1.3785890345531353 - Train Accuracy:  0.6969697
Epoch:  389 - cost:  0.54580176 

...

Epoch:  566 - cost:  0.4680289 - MSE:  1.5827426085332066 - Train Accuracy:  0.75757575
Epoch:  567 - cost:  0.72857696 - MSE:  1.6923611494711595 - Train Accuracy:  0.6121212
Epoch:  568 - cost:  0.41318262 - MSE:  1.3183323327976133 - Train Accuracy:  0.8424242
Epoch:  569 - cost:  0.39259288 - MSE:  1.3799181525841249 - Train Accuracy:  0.8666667
Epoch:  570 - cost:  0.3994569 - MSE:  1.3075598153332457 - Train Accuracy:  0.7939394
Epoch:  571 - cost:  0.4460901 - MSE:  1.6371589749383484 - Train Accuracy:  0.7939394
Epoch:  572 - cost:  0.75397754 - MSE:  1.8089174040007534 - Train Accuracy:  0.6181818
Epoch:  573 - cost:  0.4029941 - MSE:  1.35265386277907 - Train Accuracy:  0.8363636
Epoch:  574 - cost:  0.38215315 - MSE:  1.3663452201157875 - Train Accuracy:  0.8484849
Epoch:  575 - cost:  0.36107463 - MSE:  1.3936912066815919 - Train Accuracy:  0.8606061
Epoch:  576 - cost:  0.34334335 - MSE:  1.4075519607287543 - Train Accuracy:  0.8666667
Epoch:  577 - cost:  0.33077404 - MSE

Epoch:  660 - cost:  0.4659609 - MSE:  1.4023596169801946 - Train Accuracy:  0.75151515
Epoch:  661 - cost:  0.4251599 - MSE:  1.3894920591516755 - Train Accuracy:  0.7878788
Epoch:  662 - cost:  0.3879634 - MSE:  1.4077574608471164 - Train Accuracy:  0.8424242
Epoch:  663 - cost:  0.35190603 - MSE:  1.4751775128140883 - Train Accuracy:  0.8787879
Epoch:  664 - cost:  0.32417917 - MSE:  1.5649119191183378 - Train Accuracy:  0.8909091
Epoch:  665 - cost:  0.30682847 - MSE:  1.6715692118949115 - Train Accuracy:  0.8909091
Epoch:  666 - cost:  0.29529104 - MSE:  1.7694493932169058 - Train Accuracy:  0.8787879
Epoch:  667 - cost:  0.29572898 - MSE:  1.9039495553824775 - Train Accuracy:  0.8666667
Epoch:  668 - cost:  0.3822048 - MSE:  2.0276118721369327 - Train Accuracy:  0.8363636
Epoch:  669 - cost:  0.9255285 - MSE:  3.7738347032247104 - Train Accuracy:  0.6787879
Epoch:  670 - cost:  1.2201831 - MSE:  2.9444692297505632 - Train Accuracy:  0.46060607
Epoch:  671 - cost:  0.6429194 - MSE

Epoch:  754 - cost:  0.3531194 - MSE:  2.0064498387331082 - Train Accuracy:  0.8181818
Epoch:  755 - cost:  0.4125871 - MSE:  1.53924409440221 - Train Accuracy:  0.7818182
Epoch:  756 - cost:  0.30228296 - MSE:  1.9295065632155695 - Train Accuracy:  0.8424242
Epoch:  757 - cost:  0.30793393 - MSE:  1.7023186718147447 - Train Accuracy:  0.8909091
Epoch:  758 - cost:  0.3190982 - MSE:  2.4278396398044713 - Train Accuracy:  0.8242424
Epoch:  759 - cost:  0.45014736 - MSE:  2.078815711198819 - Train Accuracy:  0.74545455
Epoch:  760 - cost:  0.5413776 - MSE:  2.9658324363599773 - Train Accuracy:  0.73333335
Epoch:  761 - cost:  0.7438312 - MSE:  2.063994309761146 - Train Accuracy:  0.5030303
Epoch:  762 - cost:  0.4723028 - MSE:  0.9469837508539753 - Train Accuracy:  0.77575755
Epoch:  763 - cost:  0.37246716 - MSE:  1.3287528304656995 - Train Accuracy:  0.8484849
Epoch:  764 - cost:  0.33651718 - MSE:  1.3767119293217727 - Train Accuracy:  0.9030303
Epoch:  765 - cost:  0.31748566 - MSE: 

Epoch:  848 - cost:  0.3209806 - MSE:  2.3563761882325496 - Train Accuracy:  0.830303
Epoch:  849 - cost:  0.38506848 - MSE:  1.899775167967319 - Train Accuracy:  0.7878788
Epoch:  850 - cost:  0.36182317 - MSE:  2.6709522402323387 - Train Accuracy:  0.8121212
Epoch:  851 - cost:  0.46730494 - MSE:  1.8609452514575016 - Train Accuracy:  0.7090909
Epoch:  852 - cost:  0.2737708 - MSE:  1.8203831512199733 - Train Accuracy:  0.9030303
Epoch:  853 - cost:  0.26301616 - MSE:  1.7631441998873751 - Train Accuracy:  0.91515154
Epoch:  854 - cost:  0.25099167 - MSE:  2.147244799214589 - Train Accuracy:  0.8969697
Epoch:  855 - cost:  0.25417808 - MSE:  1.982451188007132 - Train Accuracy:  0.90909094
Epoch:  856 - cost:  0.2763767 - MSE:  2.7617149422974823 - Train Accuracy:  0.8727273
Epoch:  857 - cost:  0.3891689 - MSE:  2.137008402008515 - Train Accuracy:  0.7939394
Epoch:  858 - cost:  0.5213682 - MSE:  3.8249775246399027 - Train Accuracy:  0.76363635
Epoch:  859 - cost:  0.7878753 - MSE:  

Epoch:  942 - cost:  0.23984437 - MSE:  2.182192066067737 - Train Accuracy:  0.92727274
Epoch:  943 - cost:  0.22620776 - MSE:  2.9886270007493074 - Train Accuracy:  0.92121214
Epoch:  944 - cost:  0.22963278 - MSE:  2.3732051548542734 - Train Accuracy:  0.92121214
Epoch:  945 - cost:  0.23536673 - MSE:  3.5340040586287698 - Train Accuracy:  0.9030303
Epoch:  946 - cost:  0.29199442 - MSE:  2.4253596734334204 - Train Accuracy:  0.8848485
Epoch:  947 - cost:  0.45687953 - MSE:  5.36543617655432 - Train Accuracy:  0.77575755
Epoch:  948 - cost:  1.0753782 - MSE:  2.7617776547239266 - Train Accuracy:  0.5030303
Epoch:  949 - cost:  0.34803095 - MSE:  2.4730234150149517 - Train Accuracy:  0.8242424
Epoch:  950 - cost:  0.29616356 - MSE:  1.8805589518216375 - Train Accuracy:  0.93333334
Epoch:  951 - cost:  0.27484003 - MSE:  2.4747466014763813 - Train Accuracy:  0.92121214


In [ ]:
save_path = saver.save(sess,model_path)
print("Model Saved in fie: %s"%save_path)

In [ ]:
#Plot MSE and Accuracy graph
plt.plot(mse_history,'r')
plt.show()
plt.plot(accuracy_history)
plt.show()

In [ ]:
#Print the final Accuracy
correct_predictions = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions,tf.float32))

print("Test Accuracy: ",(sess.run(accuracy,feed_dict={x:test_x,y_:test_y})))


In [ ]:
#print the final mean square error

pred_y = sess.run(y,feed_dict={x:test_x})
mse = tf.reduce_mean(tf.square(pred_y - test_y))
print("MSE: %.4f" %sess.run(mse))